In [24]:
import pandas as pd
import datetime as dt

In [25]:
eleves = pd.read_csv('../data/DIM_ELEVE.csv', parse_dates=['DATE_NAISSANCE'])

In [26]:
print(eleves.columns)
print(eleves.shape)
print(eleves.GENRE.unique())

Index(['ID_ELEVE', 'GENRE', 'DATE_NAISSANCE', 'ID_CLASSE'], dtype='object')
(10000, 4)
['m' 'f']


In [27]:
dist_genre = eleves.groupby('GENRE').count()['ID_ELEVE']

In [28]:
reponses = pd.read_csv('../data/FACT_REPONSE.csv', parse_dates=['DATE_TIME'])

In [29]:
print(reponses.columns)
print(reponses.shape)

Index(['ID_LOG', 'KEY_REPONSE', 'ID_REPONDANT', 'DATE_TIME'], dtype='object')
(100000, 4)


In [30]:
reponses.columns

Index(['ID_LOG', 'KEY_REPONSE', 'ID_REPONDANT', 'DATE_TIME'], dtype='object')

In [31]:
merged = reponses.merge(
    eleves, left_on = "ID_REPONDANT", right_on = "ID_ELEVE"
    )

In [32]:
merged['age_group'] = pd.cut(
    # delta t annees
    (merged.DATE_TIME - merged.DATE_NAISSANCE).dt.days // 365.2425,
    bins = [0, 11, 13, 15, 17, 99],
    labels = ['0-11', '11-12', '13-14', '15-17', '>17'],
    right = False
)

In [33]:
merged.groupby("age_group").count()

/tmp/ipykernel_1670987/2799898231.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged.groupby("age_group").count()


,ID_LOG,KEY_REPONSE,ID_REPONDANT,DATE_TIME,ID_ELEVE,GENRE,DATE_NAISSANCE,ID_CLASSE
age_group,,,,,,,,
0-11,34211,34211,34211,34211,34211,34211,34211,34211
11-12,21915,21915,21915,21915,21915,21915,21915,21915
13-14,21440,21440,21440,21440,21440,21440,21440,21440
15-17,22434,22434,22434,22434,22434,22434,22434,22434
>17,0,0,0,0,0,0,0,0


In [34]:
n_eleves = eleves.shape[0]

In [35]:
reponses.KEY_REPONSE.value_counts()['B_2_1_d'] / n_eleves

0.0608

In [36]:
merged.loc[merged.KEY_REPONSE == 'B_2_1_d'].groupby("GENRE").count()['ID_LOG'] / dist_genre

GENRE
f    0.060870
m    0.060732
dtype: float64

In [37]:
# use distinct ID_ELEVE per age_group to get age_group distributions

In [38]:
merged.loc[merged.KEY_REPONSE == 'B_2_1_d'].groupby("age_group")['ID_LOG'].count()

/tmp/ipykernel_1670987/1569251425.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged.loc[merged.KEY_REPONSE == 'B_2_1_d'].groupby("age_group")['ID_LOG'].count()


age_group
0-11     203
11-12    135
13-14    148
15-17    122
>17        0
Name: ID_LOG, dtype: int64

In [39]:
merged.groupby("age_group").apply(lambda x: x.ID_REPONDANT.unique().shape[0])

/tmp/ipykernel_1670987/4267255461.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged.groupby("age_group").apply(lambda x: x.ID_REPONDANT.unique().shape[0])
/tmp/ipykernel_1670987/4267255461.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged.groupby("age_group").apply(lambda x: x.ID_REPONDANT.unique().shape[0])


age_group
0-11     1704
11-12    1103
13-14    1070
15-17    1123
>17         0
dtype: int64

In [40]:
merged[['ID_REPONDANT', "ID_ELEVE", "DATE_NAISSANCE", "DATE_TIME", 'age_group']].loc[merged.DATE_TIME.dt.year < 2026].sort_values(["ID_REPONDANT", "DATE_TIME"])


,ID_REPONDANT,ID_ELEVE,DATE_NAISSANCE,DATE_TIME,age_group
1411,E00001,E00001,2013-01-01,2025-02-01,11-12
2746,E00001,E00001,2013-01-01,2025-02-01,11-12
3813,E00001,E00001,2013-01-01,2025-02-01,11-12
14647,E00001,E00001,2013-01-01,2025-02-01,11-12
16098,E00001,E00001,2013-01-01,2025-02-01,11-12
...,...,...,...,...,...
83470,E05000,E05000,2015-01-01,2025-01-24,0-11
90780,E05000,E05000,2015-01-01,2025-01-24,0-11
91124,E05000,E05000,2015-01-01,2025-01-24,0-11
94532,E05000,E05000,2015-01-01,2025-01-24,0-11


In [41]:
merged.shape

(100000, 9)

In [42]:
reponses.shape

(100000, 4)

In [43]:
eleves.ID_ELEVE.unique().shape

(10000,)

In [44]:
reponses.ID_REPONDANT.unique().shape

(5000,)

In [45]:
merged.ID_REPONDANT.unique().shape

(5000,)

In [46]:
merged.ID_ELEVE.unique().shape

(5000,)